# Tinman FDRA Quickstart

This notebook demonstrates the core capabilities of Tinman, the Forward-Deployed Research Agent
for discovering AI reliability failures.

## Prerequisites

```bash
pip install tinman[all]
export ANTHROPIC_API_KEY=your-key-here
```

In [ ]:
# Setup and imports
import asyncio
from datetime import datetime, timedelta

from tinman import Tinman, create_tinman
from tinman.config.modes import OperatingMode
from tinman.taxonomy import FailureClass, Severity

## 1. Create a Tinman Instance

Tinman operates in three modes:
- **LAB**: Safe exploration and hypothesis testing
- **SHADOW**: Observes production traffic, no interventions
- **PRODUCTION**: Active intervention with human approval

In [ ]:
# Create Tinman in LAB mode (safe for experimentation)
async def setup_tinman():
    tinman = await create_tinman(
        mode=OperatingMode.LAB,
        skip_db=True,  # Use in-memory storage for quickstart
    )
    return tinman


tinman = await setup_tinman()
print(f"Tinman initialized in {tinman.state.mode.value} mode")
print(f"Session ID: {tinman.state.session_id}")

## 2. Run a Research Cycle

A research cycle consists of:
1. Generating hypotheses about potential failures
2. Running experiments to test hypotheses
3. Analyzing results and discovering failures
4. Proposing interventions

In [ ]:
# Run a research cycle focused on tool use failures
results = await tinman.research_cycle(
    focus="tool_use",
    max_hypotheses=3,
    max_experiments=2,
    runs_per_experiment=5,
)

print(f"\nResearch Cycle Complete!")
print(f"Hypotheses generated: {len(results.get('hypotheses', []))}")
print(f"Experiments run: {len(results.get('experiments', []))}")
print(f"Failures discovered: {len(results.get('failures', []))}")
print(f"Interventions proposed: {len(results.get('interventions', []))}")

## 3. Examine Discovered Failures

Failures are classified using the taxonomy:
- Primary class (e.g., TOOL_USE, INSTRUCTION_FOLLOWING)
- Severity (S0-S4)
- Reproducibility (0-100%)

In [ ]:
# Display discovered failures
failures = results.get("failures", [])

for failure in failures:
    print(f"\n{'=' * 60}")
    print(f"ID: {failure['id']}")
    print(f"Class: {failure['primary_class']}")
    print(f"Severity: {failure['severity']}")
    print(f"Reproducibility: {failure['reproducibility']:.0%}")
    print(f"Description: {failure['description'][:200]}...")

## 4. Interactive Discussion

You can have a conversation with Tinman to explore findings,
discuss hypotheses, or request specific analysis.

In [ ]:
# Ask Tinman about its findings
response = await tinman.discuss(
    "What's the most significant finding from your research so far? "
    "What would you recommend we focus on next?"
)

print("Tinman's analysis:")
print(response)

## 5. Generate a Report

Tinman can generate comprehensive reports of its findings.

In [ ]:
# Generate a markdown report
report = await tinman.generate_report(format="markdown")

print(report)

## 6. Check Current State

Monitor Tinman's current state and statistics.

In [ ]:
# Get current state
state = tinman.get_state()

print("Current Tinman State:")
print(f"  Mode: {state['mode']}")
print(f"  Hypotheses generated: {state['hypotheses_generated']}")
print(f"  Experiments run: {state['experiments_run']}")
print(f"  Failures discovered: {state['failures_discovered']}")
print(f"  Interventions proposed: {state['interventions_proposed']}")

## 7. Cleanup

Always close Tinman when done to release resources.

In [ ]:
# Cleanup
await tinman.close()
print("Tinman session closed.")

## Next Steps

- **02_failure_taxonomy.ipynb**: Deep dive into the failure classification system
- **03_experiments.ipynb**: Creating and running custom experiments
- **04_interventions.ipynb**: Understanding and deploying interventions
- **05_production_deployment.ipynb**: Deploying Tinman in production